UFO Dataset
------------

In [2]:
import pandas as pd 
import numpy as np

In [3]:
ufos = pd.read_csv("ufo.csv")
ufos.head(10)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611
5,10/10/1961 19:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889
6,10/10/1965 21:00,penarth (uk/wales),NaN,gb,circle,180.0,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000
7,10/10/1965 23:45,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333
8,10/10/1966 20:00,pell city,al,us,disk,180.0,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.586111,-86.286111
9,10/10/1966 21:00,live oak,fl,us,disk,120.0,several minutes,Saucer zaps energy from powerline as my pregna...,5/11/2005,30.294722,-82.984167


We are going to predict the country, given the number of seconds, latitude and longitude value. So we are going to clean the data according to that.

In [4]:
ufos = pd.DataFrame({'Seconds' : ufos['duration (seconds)'], 'Country': ufos['country'], 'Latitude': ufos['latitude'], 'Longitude': ufos['longitude']})
ufos.head(10)

,Seconds,Country,Latitude,Longitude
0,2700.0,us,29.883056,-97.941111
1,7200.0,NaN,29.384210,-98.581082
2,20.0,gb,53.200000,-2.916667
3,20.0,us,28.978333,-96.645833
4,900.0,us,21.418056,-157.803611
5,300.0,us,36.595000,-82.188889
6,180.0,gb,51.434722,-3.180000
7,1200.0,us,41.117500,-73.408333
8,180.0,us,33.586111,-86.286111
9,120.0,us,30.294722,-82.984167


We are going to make the country unique.

In [5]:
ufos.Country.unique()
ufos.dropna(inplace=True)

We are going to reduce the data even more by only importing the sightings between 1 to 120 seconds.

In [6]:
ufos = ufos[(ufos['Seconds'] >=1) & (ufos['Seconds'] <= 120)]
ufos.head()

,Seconds,Country,Latitude,Longitude
2,20.0,gb,53.200000,-2.916667
3,20.0,us,28.978333,-96.645833
9,120.0,us,30.294722,-82.984167
14,30.0,us,35.823889,-80.253611
16,120.0,us,42.537778,-83.233056


We are going to encode the data for using Scikit's Label Encoder.

In [7]:
from sklearn.preprocessing import LabelEncoder
ufos['Country'] = LabelEncoder().fit_transform(ufos['Country'])
ufos.head()

,Seconds,Country,Latitude,Longitude
2,20.0,3,53.200000,-2.916667
3,20.0,4,28.978333,-96.645833
9,120.0,4,30.294722,-82.984167
14,30.0,4,35.823889,-80.253611
16,120.0,4,42.537778,-83.233056


In [8]:
# Model Training

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
Selected_features = ['Seconds', 'Latitude', 'Longitude']

X = ufos[Selected_features]
y = ufos['Country']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, predictions))

Accuracy:  0.9587691374867364


D:\Workstation\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Now the model needs to be 'pickled'

In [14]:
import pickle
model_filename = 'ufo-model.pkl'
pickle.dump(model,open(model_filename,'wb'))

model = pickle.load(open('ufo-model.pkl','rb'))
print(model.predict([[50,44,-12]]))

[3]


'3' is the country code for UK.